## FDW Crop Production Data Profiling - Sudan

In [ ]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [ ]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('./data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'EPSG:20135'
fn_shapes = sorted(glob.glob('./data/shapefile/fewsnet/SD_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
#read in username and password
exec(open('/Users/wanders7/Documents/Code/Logins/FDW.py').read())
auth = (username, password)
parameters = {
    'format': 'json',
    'country': 'Sudan',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

In [ ]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'

#Drop the blank (nan) rows that are duplicates of rows that do have data
df = df[np.isfinite(df.value)]

#drop the admin 0 observations
df = df[~(df.fnid.astype(str).str[6:8]=='A0')]

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([SD_Admin1_1976,SD_Admin1_1990,SD_Admin1_2005,SD_Admin1_2011,SD_Admin1_2012,SD_Admin1_2013,SD_Admin1_2014], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

## Sudan
Sudan crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-sudan.png?itok=DRc-ECNF)

![USDA](https://ipad.fas.usda.gov/countrysummary/images/SU/cropcalendar/eafrica_su_calendar.png)

- comparison of boundaries
![image](./figures/SD_admin_shapes.png)

- In Sudan, basd on the FEWS NET data, there were six changes in administrative units, although they are mostly minor at the admin 1 level

| year	 | Admin1   | # units   | Admin2   | # units   |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1976	 | SD1976A1 | 18	| nan	| 0	|
| 1990	 | SD1990A1 | 26	| nan	| 0	|
| 2005	 | SD2005A1 | 25	| nan	| 0	|
| 2011	 | SD2011A1 | 15	| SD2011A2	| 133	|
| 2012	 | SD2012A1 | 17	| nan	| 0	|
| 2013	 | SD2013A1 | 18	| nan	| 0	|
| 2014	 | SD2014A1 | 18	| SD2014A2	| 172	|


- Only wheat has two seasons (Main and Winter), and the vast majority of data is winter
- While there is a shapefile for 2021, there is no crop data associated with the 2021 shapefile, so we use 2014 as the most recent shapefile

In [ ]:
# Define the latest shapefile ------------------- #
latest_level = 1
shape_latest = SD_Admin1_2014.copy().to_crs('epsg:20135')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [ ]:
# Link admin boundaries ------------------------- #
link_1976, over_1976 = FDW_PD_CreateAdminLink(SD_Admin1_1976, SD_Admin1_2013, 'ADMIN1', 'ADMIN1', prod, epsg)
link_1990, over_1990 = FDW_PD_CreateAdminLink(SD_Admin1_1990, SD_Admin1_2013, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2005, over_2005 = FDW_PD_CreateAdminLink(SD_Admin1_2005, SD_Admin1_2013, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2011, over_2011 = FDW_PD_CreateAdminLink(SD_Admin1_2011, SD_Admin1_2013, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2012, over_2012 = FDW_PD_CreateAdminLink(SD_Admin1_2012, SD_Admin1_2013, 'ADMIN1', 'ADMIN1', prod, epsg)

# Crop specific ratios
link_ratio_1976 = FDW_PD_RatioAdminLink(link_1976, prod, over_1976, mdx_pss)
link_ratio_1990 = FDW_PD_RatioAdminLink(link_1990, prod, over_1990, mdx_pss)
link_ratio_2005 = FDW_PD_RatioAdminLink(link_2005, prod, over_2005, mdx_pss)
link_ratio_2011 = FDW_PD_RatioAdminLink(link_2011, prod, over_2011, mdx_pss)
link_ratio_2012 = FDW_PD_RatioAdminLink(link_2012, prod, over_2012, mdx_pss)

# Merge link_ratio
assert link_ratio_1976.keys() == link_ratio_2012.keys()
assert link_ratio_1990.keys() == link_ratio_2012.keys()
assert link_ratio_2005.keys() == link_ratio_2012.keys()
assert link_ratio_2011.keys() == link_ratio_2012.keys()

link_merged = [link_ratio_1976, link_ratio_1990, link_ratio_2005, 
               link_ratio_2011,link_ratio_2012]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #

In [ ]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "growing year"
cols = ['season_name','product','crop_production_system','growing_month','harvest_year','harvest_month','growing_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','growing_year','growing_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack_gscd = stack[names]
stack_gscd['gscd_code'] = 'calibrated'
# ----------------------------------------------- #

# Reported FDW data ----------------------------- #
stack_fdw = df[names]
stack_fdw['indicator'] = stack_fdw['indicator'].replace({'Area Harvested':'area','Quantity Produced':'production','Yield':'yield'})
stack_fdw['gscd_code'] = 'reported'
# ----------------------------------------------- #

# Final Processing ------------------------------ #
stack = pd.concat([stack_fdw,stack_gscd], axis=0).reset_index(drop=True)
# No concerns found for grain types
stack['product'] = stack['product'].replace(product_category)
# Calibration of Growing and Harvest year and season
cs = {
    'Main': {'growing_month':{'10-01':'10-01'},'harvest_month':{'10-01':'10-01'}},
    'Winter': {'growing_month':{'04-01':'04-01'},'harvest_month':{'04-01':'04-01'}},
}

cy = {}
stack, df, link_ratio = FDW_PD_CaliSeasonYear(stack, df, link_ratio, cs, cy)
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# # ----------------------------------------------- #

# Save data
save_hdf('./data/crop/adm_crop_production_SD.hdf', stack)
save_hdf('./data/crop/adm_crop_production_SD_raw.hdf', df)
save_npz('./data/crop/adm_crop_production_SD_ratio.npz', link_ratio)

## Visualization of production data

In [ ]:
# Bar chart of national grain production
country_iso, country_name = 'SD', 'Sudan'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Wheat Grain','Millet', 'Sorghum']
for season_name in ['Main','Winter']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = './figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    fig.show()

In [ ]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name ='SD', 'Sudan'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Millet','Main'],
    ['Sorghum','Main'],
    ['Wheat','Winter'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = './figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()

In [ ]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'SD', 'Sudan'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
link_ratio = load_npz('./data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
product_season = [
    ['Millet','Main'],
    ['Sorghum','Main'],
    ['Wheat','Winter'],
]
for product_name, season_name in product_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = './figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)